In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
data = pd.read_csv('./data/weatherHistory.csv')
data_end = int(np.floor(0.8*(data.shape[0])))
dataset = data.iloc[:, 3:4].values
dataset_train = dataset[:data_end]
dataset_test = dataset[data_end:]
dataset_train

array([[ 9.47222222],
       [ 9.35555556],
       [ 9.37777778],
       ...,
       [ 1.16111111],
       [-0.11111111],
       [-0.94444444]])

In [7]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(dataset_train)
training_set_scaled

array([[0.50697507],
       [0.50508505],
       [0.50544505],
       ...,
       [0.37233372],
       [0.35172352],
       [0.33822338]])

In [9]:
X_train = []
y_train = []
for i in range(24, dataset_train.shape[0]):
    X_train.append(training_set_scaled[i-24:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train

array([[[0.50697507],
        [0.50508505],
        [0.50544505],
        ...,
        [0.53469535],
        [0.51741517],
        [0.51876519]],

       [[0.50508505],
        [0.50544505],
        [0.48780488],
        ...,
        [0.51741517],
        [0.51876519],
        [0.52236522]],

       [[0.50544505],
        [0.48780488],
        [0.49536495],
        ...,
        [0.51876519],
        [0.52236522],
        [0.51408514]],

       ...,

       [[0.38205382],
        [0.38151382],
        [0.3902439 ],
        ...,
        [0.37188372],
        [0.38169382],
        [0.3798038 ]],

       [[0.38151382],
        [0.3902439 ],
        [0.3898839 ],
        ...,
        [0.38169382],
        [0.3798038 ],
        [0.37233372]],

       [[0.3902439 ],
        [0.3898839 ],
        [0.3904239 ],
        ...,
        [0.3798038 ],
        [0.37233372],
        [0.35172352]]])

In [11]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [12]:
# Initialising the RNN
regressor = Sequential() # lưu trữ các danh sách 

# Adding the first LSTM layer and some Dropout regularisation
#Dropout bỏ qua 20% unit bất kì
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

In [13]:
# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [14]:
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [15]:
# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
#thuật toán tối ưu là thuật toán để cải thiện weight và bias theo từng bước
#adam thuật toán tối ưu tìm cực tiểu toàn cục

In [16]:
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)
#Epochs: Một Epoch được tính là khi chúng ta đưa tất cả dữ liệu trong tập train vào mạng neural network 1 lần
#Batch size là số lượng mẫu dữ liệu trong một lần huấn luyện.

Epoch 1/100
2411/2411 [==============================] - 240s 90ms/step - loss: 0.0280
Epoch 2/100
2411/2411 [==============================] - 252s 104ms/step - loss: 0.0251
Epoch 3/100
2411/2411 [==============================] - 261s 108ms/step - loss: 0.0249
Epoch 4/100
1936/2411 [=======================>......] - ETA: 54s - loss: 0.0248